In [35]:
import load_graph
import pandas as pd
import numpy as np

In [36]:
edgelist = pd.read_csv('Data/ForAnalysis/ConsensusPathDB_human_PPI_HiConfidence_snap.csv')
x = pd.read_csv('Data/ForAnalysis/ConsensusPathDB_human_PPI_HiConfidence_snap_FeatsMat.csv')
x.drop(columns=x.columns[0], inplace=True)

In [37]:
assert len(x['index'].unique()) == len(x['index']) # no repeated features

In [38]:
# find all unique nodes
nodes = edgelist['node0'].append(edgelist['node1'])
nodes = nodes.unique()
mappings = { nodes[i]: i for i in range(len(nodes)) }

In [39]:
normalized_edgelist = set()
for index, row in edgelist.iterrows():
    if tuple([mappings[row['node0']], mappings[row['node1']]]) in normalized_edgelist:
        print('no')
    normalized_edgelist.add(tuple([mappings[row['node0']], mappings[row['node1']]]))
    if tuple([mappings[row['node1']], mappings[row['node0']]]) in normalized_edgelist:
        print('no')
    normalized_edgelist.add(tuple([mappings[row['node1']], mappings[row['node0']]]))

In [42]:
normalized_node0 = edgelist['node0'].apply(lambda cell: mappings[cell])
normalized_node1 = edgelist['node1'].apply(lambda cell: mappings[cell])
normalized_edgelist = pd.concat((normalized_node0, normalized_node1), axis=1)
backwards_edges = pd.concat((normalized_edgelist['node0'], normalized_edgelist['node1']), axis=1)
normalized_edgelist = pd.concat((normalized_edgelist, backwards_edges))

In [44]:
len(nodes)

14011

In [45]:
len(x)

6364

In [46]:
known_genes = pd.read_csv('Data/ForAnalysis/Y/NCG_cancergenes_list.txt', sep='\t')
known = set(known_genes['711_Known_Cancer_Genes'])

In [48]:
y = np.zeros(len(nodes))
feature_matrix = pd.DataFrame(np.zeros((len(nodes), 3)), dtype=float, columns=['breast', 'bladder', 'kidney'])
for i, row in x.iterrows():
    new_index = mappings[row['index']]
    feature_matrix.iloc[new_index] = row[['breast', 'bladder', 'kidney']]
    y[new_index] = int(row['Hugo_Symbol'] in known)

In [49]:
feature_matrix.to_csv('feature_matrix.csv')

In [50]:
np.savetxt('y.txt', y)

In [51]:
normalized_edgelist.to_csv('edgelist.csv')

In [52]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thurs Nov 7 21:27:10 2019

@author: Kendrick

util functions
"""
import pandas as pd
import numpy as np

import torch
from torch_geometric.data import Data, DataLoader

def load_pyg(x, edge_index, y):
    X = pd.read_csv(x)
    X.drop(columns=X.columns[0], inplace=True)
    X = torch.tensor(X.values, dtype=torch.float)
    
    edges = pd.read_csv(edge_index)
    edges.drop(columns=edges.columns[0], inplace=True)
    edges = torch.tensor(edges.values, dtype=torch.long)
    
    Y = np.loadtxt(y)
    
    data = Data(x=X, edge_index=edges.t().contiguous(), y=Y)
    return data

In [53]:
data = load_pyg('feature_matrix.csv', 'edgelist.csv', 'y.txt')

In [54]:
data

Data(edge_index=[2, 496434], x=[14011, 3], y=[0. 0. 0. ... 0. 0. 0.])